In [1]:
%matplotlib inline
import rtsa
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from scipy.ndimage import label, find_objects
from io import BytesIO

In [2]:
import os
def find_all_rtsa_files():
    files = []
    for x in os.listdir():
        if x.endswith(".rtsa"):
            files.append(x)
    return files

In [3]:
class Result:
    def __init__(self,box,samples, contact, car_num):
        time_slice, frequency_slice = box #box will be a binary mask, e.g a slice with times and frequencies matching the mask
        #to get from time_slice to the correct time, we need to calculate based on the linspace in get_heatmap_png
        #yticks = np.linspace(0, samples.samples.shape[0], 10)
        #its an array from 0 to 93 in 10 steps
        #in order to remap this, we need the starting time + "the whole time" (=time intervall) times the number of time steps (=time_slice) normalized
        #because that way the 0. step will give the starting time, the last step will give the end time,...
        time_arr_size = samples.samples.shape[0]
        start_time = samples.samples_start_time
        whole_time = samples.samples_end_time - samples.samples_start_time
        start_time_in_seconds = start_time + whole_time * (time_slice.start/time_arr_size)
        end_time_in_seconds = start_time + whole_time * (time_slice.stop/time_arr_size)
        #the same for the frequency, just that the array now has around 82000 values
        #xticks_labels = list(np.linspace(samples.frequency_start / 1e6, (samples.frequency_start + samples.frequency_span) / 1e6, 10))
        frequency_arr_size = samples.samples.shape[1]
        start_frequency = samples.frequency_start/1e6
        whole_frequency = samples.frequency_span/1e6 
        start_frequency_in_mhz = start_frequency + whole_frequency * (frequency_slice.start/frequency_arr_size)
        end_frequency_in_mhz = start_frequency + whole_frequency * (frequency_slice.stop/frequency_arr_size)
        #class properties
        self.start_time = start_time_in_seconds
        self.end_time = end_time_in_seconds
        self.start_frequency = start_frequency_in_mhz
        self.end_frequency = end_frequency_in_mhz
        self.max_power = np.max(samples.samples[time_slice,frequency_slice])
        self.contact = contact
        self.car_num=car_num
    
    #https://stackoverflow.com/questions/10252010/serializing-class-instance-to-json
    def convert_to_json(self):
        dic = {"contact": self.contact,
                "car_num" : self.car_num,
               "frq_start" : self.start_frequency,
               "frq_end" : self.end_frequency,
               "time_start" : self.start_time,
               "time_end" : self.end_time,
               "max_power" : float(self.max_power) #TypeError: Object of type float32 is not JSON serializable
              }
        return json.dumps(dic)
        

In [4]:
def find_carriers(samples, filename):
    threshold = 3.5  #Number of std away from mean -> higher threshold, more carriers get detected
    mean = np.mean(samples.samples)
    std = np.std(samples.samples)
    #https://stackoverflow.com/questions/22103572/how-to-find-the-rectangles-in-an-image
    binary_mask = np.abs(samples.samples-mean)>(threshold*std)
    labeled_array, n = label(binary_mask)
    bboxes = find_objects(labeled_array)
    carriers_info = []
    count = 0
    for box in tqdm(bboxes):
        count = count+1
        carrier_info = Result(box, samples, filename, count)
        carriers_info.append(carrier_info)
    return carriers_info

In [5]:
files = find_all_rtsa_files()
samples_list = []
for file in files:
    with open(file, "rb") as file_object:
            #rtsa.print_file_chunks(file_object)
            samples = rtsa.get_samples_from_file(file_object)
            samples_list.append(samples)
    heatmap_bytes = rtsa.get_heatmap_png(samples)
    carriers = find_carriers(samples, file)
    s = json.dumps([c.convert_to_json() for c in carriers])
    with open(f"{file}-samples2.json", "w") as j: #only w because it is no binary
        j.write(s)
    with open(f"{file}-samples.png", "wb") as heatmap_file:
        heatmap_file.write(heatmap_bytes)
    

get_samples_array: Number of samples: 7052000
get_samples_array: Number of sweeps: 86.0


100%|███████████████████████████████████████████████████████| 2345/2345 [00:00<00:00, 173083.50it/s]


get_samples_array: Number of samples: 6970000
get_samples_array: Number of sweeps: 85.0


100%|███████████████████████████████████████████████████████| 2283/2283 [00:00<00:00, 199974.86it/s]


get_samples_array: Number of samples: 7790000
get_samples_array: Number of sweeps: 95.0


...

100%|███████████████████████████████████████████████████████| 6219/6219 [00:00<00:00, 212723.57it/s]


get_samples_array: Number of samples: 6724000
get_samples_array: Number of sweeps: 82.0


100%|███████████████████████████████████████████████████████| 9659/9659 [00:00<00:00, 206088.02it/s]


get_samples_array: Number of samples: 3772000
get_samples_array: Number of sweeps: 46.0


100%|███████████████████████████████████████████████████████| 1405/1405 [00:00<00:00, 192644.56it/s]
